<font color= "lime"> <h1> Medical chatbot using "DistilBERT" model

<font color="orange"> <i> the main idea is to use a pre trined transformer model for a question answering medical chatbot. we cannot build the bot from scratch because first of all we need a huge amount of data for training the bot. collecting that much data is nearly impossible. even if we can manage that mucch huge data, we don't have that much GPU power for training the model using that much data. <br>

<font color="orange"> <i>so we have used a model that is already trained on a huge amout of data. Now we will just do fine tuning the model using our small dataset. Again using larnge amount of data for fine tuning will always increase the accuracy of the bot. So the more information we will provide to the bot the more it will learn

<font color="lime"> making weights constant for repetative result

In [ ]:
import numpy as np
import random as rn

In [ ]:
import os
os.environ['PYTHONHASHSEED']= '0'

In [ ]:
np.random.seed(0)

In [ ]:
rn.seed(0)

<font color="lime"> Importing

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
import unicodedata
import re
import os
import io
import time

import warnings
warnings.filterwarnings('ignore')

<font color= "lime"> Importing pre-trined transformer

In [ ]:
!pip install transformers==2.8.0
!pip install torch==1.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 563 kB 14.8 MB/s 
     |████████████████████████████████| 5.6 MB 12.6 MB/s 
     |████████████████████████████████| 132 kB 43.2 MB/s 
     |████████████████████████████████| 880 kB 64.9 MB/s 
     |████████████████████████████████| 1.3 MB 64.7 MB/s 
     |████████████████████████████████| 9.2 MB 40.9 MB/s 
     |████████████████████████████████| 79 kB 5.7 MB/s 
     |████████████████████████████████| 140 kB 47.6 MB/s 
     |████████████████████████████████| 127 kB 45.7 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=d2646ab5207a5c3eccf2f104b5ebdf26f02c44a1ca67ac6fd6cb75bae14bd2ce
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    

<font color= "lime"> Importing DistilBERT from transformer

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering
import torch



<font color="lime"> Tokenizing

In [ ]:
tokenizer= DistilBertTokenizer.from_pretrained('distilbert-base-uncased', return_token_type_ids= True)


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

<font color="lime"> selecting the distilbert-base-uncased model

In [ ]:
# some common state of the art pretrained models for NLP problems

model_type="distilbert"

model_name= "bert-base-cased"
if model_type == "bert":
    model_name = "bert-base-cased"

elif model_type == "roberta":
    model_name = "roberta-base"

elif model_type == "distilbert":
    model_name = "distilbert-base-cased"

elif model_type == "distilroberta":
    model_type = "roberta"
    model_name = "distilroberta-base"

elif model_type == "electra-base":
    model_type = "electra"
    model_name = "google/electra-base-discriminator"

elif model_type == "electra-small":
    model_type = "electra"
    model_name = "google/electra-small-discriminator"

elif model_type == "xlnet":
    model_name = "xlnet-base-cased"

In [ ]:
model= DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased-distilled-squad')


Downloading:   0%|          | 0.00/451 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/265M [00:00<?, ?B/s]

<font color="lime"> Importing the context of medical topic

In [ ]:
from google.colab import files
uploaded = files.upload()

<font color= "lime"> <b> We are using raw text as input .txt. no annotation no label. just plain text like a paragraph

In [ ]:
context = open('chatBOT_raw.txt','r').read()

In [ ]:
# printing the whole medical context
context

<font color= "lime"> In this cell we have defined a function that can  find the encoded value of the tokens and then we have used fine tuning the DistilBERT model using out medical context. then when a question is asked by user then that question is also being tokenized and our model will match the tocken of question with the whole context and will give the reply using tokens. Then those tokens will be converted into string so that user can have a meaningful reply

In [ ]:
def Question_answer(Q):

  encoding= tokenizer.encode_plus(Q, context)

  input_ids, attention_mask= encoding["input_ids"], encoding["attention_mask"]
  start_scores, end_scores= model(torch.tensor([input_ids]), attention_mask=torch.tensor([attention_mask]))


  ans_tokens= input_ids[torch.argmax(start_scores) : torch.argmax(end_scores)+1]
  answer_tokens= tokenizer.convert_ids_to_tokens(ans_tokens, skip_special_tokens= True)

  answer_tokens_to_string= tokenizer.convert_tokens_to_string(answer_tokens)

  return answer_tokens_to_string

<font color="lime"> Finally we are using a infinite loop that will continue until user enter "EXIT" this loop will take user input as question, then the question will be passed into the model and a reply will be printed out

In [ ]:
while (True):
  Q= input()

  if (Q=="EXIT"):
    break

  else:
    reply= Question_answer(Q)
    print ("BOT: ",reply)
